# L6d: Another Look at Solving Linear Equations
In this activity, we will revisit the solution of linear algebraic equations, where we will compare the solution that we obtain by using QR decomposition with our iterative methods for solving linear equations.

> __Learning Objectives__ 
>
> * You will compare direct (QR decomposition) and iterative methods (Jacobi, Gauss-Seidel, SOR) for solving linear systems using a manufactured 1D Poisson equation test problem. 
> * You will analyze the accuracy, convergence behavior, and computational performance of these different approaches to understand when each method is most appropriate.

Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on Julia functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's setup our code environment:

In [2]:
include(joinpath(@__DIR__, "Include.jl"));

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl), check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types and data used in this material. 

___

## Task 1: Setup the System of Linear Equations
In this task, we will set up a system of linear equations that we will solve using both QR decomposition and iterative methods. We will define the coefficient matrix $\mathbf{A}$ and the right-hand side vector $\mathbf{b}$ for the system of equations $\mathbf{A}\mathbf{x} = \mathbf{b}$.

### The Test Problem: 1D Poisson Equation
We're solving a classic problem: the **one-dimensional Poisson equation** with **Dirichlet boundary conditions**. This shows up everywhere in physics and engineering (heat conduction, electrostatics, fluid flow, you name it).

Here's the clever part: instead of solving an unknown problem, we work backwards with a "manufactured solution." We pick $u(x) = \sin(\pi x)$ as our known answer, discretize the domain into $n$ grid points at $x_i = \frac{i}{n+1}$, and use finite differences to approximate derivatives. The continuous equation $-u''(x) = f(x)$ becomes the discrete system $-\frac{u_{i-1} - 2u_i + u_{i+1}}{h^2} = f_i$.

**Here's the key insight**: In the manufactured solution approach, we don't actually need to know what $f(x)$ is! Since we already decided that $u(x) = \sin(\pi x)$ is our solution, we can just compute $\mathbf{b} = \mathbf{A} \mathbf{x}_{true}$ directly. The vector $\mathbf{b}$ represents the discrete version of $f(x)$ that would produce our chosen solution. It's like saying "if this is the answer, what must the question have been?"

This gives us a beautiful **tridiagonal system** $\mathbf{A}\mathbf{x} = \mathbf{b}$ where $\mathbf{A}$ has 2's on the diagonal and -1's on the off-diagonals. Since we know the exact answer, we can measure how accurate our different numerical methods actually are!

Let's setup our system of equations.

In [16]:
A,b,x_true = let 
    
    n = 100; # how big of a system do we want to solve?

    # Build the tridiagonal matrix A and the manufactured solution x⋆ and right-hand side b
    # for the 1D Poisson equation with Dirichlet boundary conditions.
    # The matrix A is constructed using the sparse diagonal matrix constructor `spdiagm`.
    di = fill(2.0, n); dl = fill(-1.0, n-1); du = fill(-1.0, n-1)
    A  = diagm(-1 => dl, 0 => di, 1 => du)              # tridiagonal matrix A  
    xg = range(1, n; step=1) ./ (n+1)                   # grid i/(n+1)
    xstar = @. sin(pi * xg)                             # "true" solution
    b = A * xstar                                       # manufactured RHS

    A,b, xstar # return
end;

What does $\mathbf{A}$ look like for this problem? (the tridiagonal structure should be apparent in the matrix representation).

In [45]:
A

100×100 Matrix{Float64}:
  2.0  -1.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   2.0  -1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   2.0  -1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   2.0  -1.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   2.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   2.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  ⋮                             ⋮    ⋱         ⋮                      
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.

### Check: Is the system matrix $\mathbf{A}$ strictly diagonally dominant?
Before we continue, let's verify the generated system matrix $\mathbf{A}$ is actually diagonally dominant.
> __Test:__ We'll compute the sum of the absolute values of each row (excluding the diagonal element), and compare it to the absolute value of the diagonal element in that row. If the sum of the absolute values of the non-diagonal elements is less than the absolute value of the diagonal element, then the matrix is diagonally dominant.

Is the matrix $\mathbf{A}$ strictly diagonally dominant?

In [14]:
ddcondition = let
    
    # initialize -
    number_of_rows = size(A, 1);
    ddcondition = Array{Bool,1}(undef, number_of_rows);

    # let's check each row
    for i ∈ 1:number_of_rows
        aii = abs(A[i,i]);
        σ = 0.0;
        for j ∈ 1:number_of_rows
            if (i ≠ j)
                σ += abs(A[i,j]);
            end
        end
        ddcondition[i] = (aii > σ) ? true : false; # ternary operator, nice!
    end

    ddcondition
end;

The `ddcondition` array contains boolean values indicating whether each row of our system matrix $\mathbf{A}$ satisfies the diagonal dominance condition. Each element `ddcondition[i]` is `true` if the absolute value of the diagonal element in row `i` is greater than the sum of the absolute values of all other elements in that row, and `false` otherwise.

> __Test:__ The assertion `@assert any(ddcondition)` checks that at least one row (and hopefully all rows) satisfies the diagonal dominance condition. If the condition fails, a warning is issued indicating that the matrix may not be suitable for iterative methods, as convergence is not guaranteed.

For diagonally dominant matrices, iterative methods like Jacobi, Gauss-Seidel, and SOR are guaranteed to converge, making them reliable choices for solving linear systems.

In [15]:
try
    @assert any(ddcondition)
catch
    @warn "Diagonal dominance condition not satisfied"
end

### Check: Is our test solution correct for the system of equations?
We generated a manufactured solution vector `x_true` and computed the corresponding right-hand side vector `b` using the system matrix `A`. Now, we will verify that our generated solution is indeed correct by checking if the equation `A * x_true = b` holds true.

> __Test:__ We will compute the product of the system matrix `A` and the solution vector `x_true`, and compare it to the right-hand side vector `b`. If they are (approximately) equal, then our generated solution is correct. We will use [the `isapprox(...)` function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to check for approximate equality in combination with [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to ensure that the test passes.

So what do we see?

In [20]:
let

    # initialize -
    atol = 1e-10; # absolute tolerance for checking the residual norm against zero

    # compute the residual for the manufactured solution
    r = b - A*x_true # if r ≈ 0, then x_true is a solution to the system of equations Ax = b
    norm_r = norm(r); # p = 2 norm of the residual vector
    
    # run the test -
    @assert isapprox(norm_r, 0.0; atol = atol);
end

If we get through without any warnings or assertion errors, then our generated solution is indeed correct for the system of equations defined by the matrix `A` and the vector `b`!
___

## Task 2: Compare the QR decomposition solution with the true solution
In this task, we will solve the system of linear equations using QR decomposition and compare the solution we obtain with the true solution vector `x_true` that we generated earlier.

> __QR Decomposition__
>
> The QR decomposition can be used as a __direct method__ for solving linear systems. The QR decomposition factors a matrix into two orthogonal matrices, $\mathbf{Q}$ and $\mathbf{R}$, such that $\mathbf{A} = \mathbf{Q} \mathbf{R}$. 
> This is super handy for solving linear systems of equations:
>$$
>\begin{align*}
>\mathbf{A} \mathbf{x} &= \mathbf{b} \\
>\mathbf{Q} \mathbf{R} \mathbf{x} &= \mathbf{b} \\
>\mathbf{R} \mathbf{x} &= \mathbf{Q}^{\top} \mathbf{b} \\
>\mathbf{x} &= \mathbf{R}^{-1} \mathbf{Q}^{\top} \mathbf{b}\quad\blacksquare
>\end{align*}
>$$
> QR decomposition is particularly useful for solving linear systems because it avoids the need to compute the inverse of the matrix $\mathbf{A}$ directly, which can be computationally expensive and numerically unstable.

Let's compute the QR decomposition of the matrix `A`, solve for the solution vector `x_qr`, and then compare it to the true solution vector `x_true` using the `isapprox(...)` function to check for approximate equality.

In [22]:
x_qr = let

    Q,R = qr(A);
    x_qr = R \ (transpose(Q) * b);

    x_qr
end;

Are the solutions `x_qr` and `x_true` approximately equal?

In [24]:
let

    # initialize -
    atol = 1e-10; # absolute tolerance for checking the residual norm against zero

    # compute the norm of the difference between the computed solution and the manufactured solution
    r = x_qr - x_true; # if r ≈ 0, then x_qr is approximately equal to x_true
    norm_r = norm(r); # p = 2 norm of the residual vector
    
    # run the test -
    @assert isapprox(norm_r, 0.0; atol = atol); # test
end

Ok! So QR decomposition gives us a solution that is approximately equal to the true solution we generated earlier. This confirms that our QR decomposition approach is valid for solving the system of linear equations defined by the matrix `A` and the vector `b`.

How about our iterative methods? Do they also give us a solution that is approximately equal to the true solution `x_true`? Let's check that next!
___

## Task 3: Compare the iterative method solutions with the true solution
In this task, we will solve the system of linear equations using the iterative methods we have implemented (Jacobi, Gauss-Seidel, and SOR) and compare the solutions we obtain with the true solution vector `x_true` that we generated earlier.

Let's setup a code block that allows us to compute the solution using an iterative method of our choice, and then compare the solution to the true solution vector `x_true` using the `isapprox(...)` function to check for approximate equality.

In [37]:
iterative_solution_archive = let

    # initialize -
    number_of_rows = size(A, 1);
    maximum_number_of_iterations = 25000; # maximum number of iterations for the iterative method
    tolerance = 1e-12;
    algorithm = GaussSeidelMethod(); # change this to Jacobi(), GaussSeidelMethod() or SuccessiveOverRelaxationMethod() to test other algorithms
    ω = 0.6; # relaxation factor, only used for SOR
    xₒ = 0.1*ones(number_of_rows); # initial solution guess xₒ

    # call the solve method with the appropriate parameters -
    x = VLDataScienceMachineLearningPackage.solve(A, b, xₒ; algorithm = algorithm, 
        maxiterations = maximum_number_of_iterations, 
        ϵ = tolerance, 
        ω = ω # only used for SOR
    );

    x; # return the solution vector computed by the iterative method
end

Dict{Int64, Vector{Float64}} with 23271 entries:
  11950 => [0.0310996, 0.0621691, 0.0931785, 0.124098, 0.154897, 0.185546, 0.21…
  1703  => [0.0257491, 0.0514786, 0.0771633, 0.102779, 0.1283, 0.153701, 0.1789…
  12427 => [0.0310997, 0.0621693, 0.0931788, 0.124098, 0.154897, 0.185547, 0.21…
  7685  => [0.0310835, 0.0621369, 0.0931302, 0.124033, 0.154817, 0.18545, 0.215…
  18374 => [0.0310999, 0.0621696, 0.0931793, 0.124099, 0.154898, 0.185548, 0.21…
  3406  => [0.0300701, 0.0601121, 0.0900969, 0.119996, 0.149779, 0.179419, 0.20…
  1090  => [0.0214191, 0.0428268, 0.0642024, 0.0855252, 0.106774, 0.12793, 0.14…
  2015  => [0.0271434, 0.0542644, 0.0813367, 0.108334, 0.135231, 0.162, 0.18861…
  18139 => [0.0310999, 0.0621696, 0.0931793, 0.124099, 0.154898, 0.185548, 0.21…
  17088 => [0.0310999, 0.0621696, 0.0931793, 0.124099, 0.154898, 0.185548, 0.21…
  11280 => [0.0310994, 0.0621686, 0.0931778, 0.124097, 0.154896, 0.185545, 0.21…
  16805 => [0.0310999, 0.0621696, 0.0931793, 0.124099, 0.154

### Check: Is the iterative solution actually a valid solution for the system of equations?
In this check, we will compute the residual vector `r` for the iterative solution obtained from our iterative method and compare its norm to zero. The residual vector is defined as `r = b - A * x_iterative`, where `x_iterative` is the solution obtained from the iterative method. 

> __Test:__ If the norm of the residual vector is approximately zero, then the iterative solution is a valid solution for the system of equations. Let's setup a code block that computes the residual vector and checks if its norm is approximately zero using [the `isapprox(...)` function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) in combination with [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to ensure that the test passes.

So what happens?

In [38]:
let

    # initialize -
    atol = 1e-10; # absolute tolerance
    i = keys(iterative_solution_archive) |> collect |> sort |> last; # get the last key (the last iteration)
    x_iterative = iterative_solution_archive[i]; # get the last solution (converged solution)

    # compute the residual for the iterative solution
    r = b - A*x_iterative; # if r ≈ 0, then x_iterative is a solution to the system of equations Ax = b
    norm_r = norm(r); # p = 2 norm of the residual vector

    # check: our solution vs Julia solution, should be approximately equal
    @assert isapprox(norm_r, 0, atol=atol)
end

So our iterative method generated a valid solution (if no warnings or assertion errors were raised), but is it approximately equal to the true solution `x_true` that we generated earlier? And, how do the choices of absolute tolerance `atol`, the maximum number of iterations and the choice of method affect the validity of the solution obtained from the iterative method?

Let's check that next!

> __Experiment:__ Compare the norm of the residual vector `r` between the true and iterative solutions with zero for different values of the absolute tolerance `atol`, maximum number of iterations, and different iterative methods (Jacobi, Gauss-Seidel, and SOR). How do these choices influence the validity of the solution obtained from the iterative method?

What do we get?

In [42]:
let

    # initialize -
    atol = 1e-8; # absolute tolerance for checking the residual norm against zero
    i = keys(iterative_solution_archive) |> collect |> sort |> last; # get the last key (the last iteration)
    x_iterative = iterative_solution_archive[i]; # get the last solution (converged solution)

    # compute the norm of the difference between the computed solution and the manufactured solution
    r = x_iterative - x_true; # if r ≈ 0, then x_iterative is approximately equal to x_true
    norm_r = norm(r); # p = 2 norm of the residual vector
    
    # run the test -
    @assert isapprox(norm_r, 0.0; atol = atol); # test
end

__Interesting!__ It appears that the choice of absolute tolerance `atol` (and the maximum number of iterations) can significantly affect the quality of the solution obtained from the iterative method.

> __Observation (atol):__ For small absolute tolerances (e.g., `atol = 1e-10`), the iterative solution is very close to the true solution, while for larger absolute tolerances (e.g., `atol = 1e-6`), the iterative solution may deviate more from the true solution. This indicates that the choice of `atol` plays a crucial role in determining the accuracy of the iterative solution. 

> __Observation (max iterations):__ The maximum number of iterations allowed for the iterative method also influences the quality of the solution. For small `atol` parameter values, a larger number of iterations may be required to achieve convergence, while for larger `atol` values, fewer iterations may be sufficient to reach a solution that is considered valid.

In summary, both the absolute tolerance `atol` and the maximum number of iterations are important factors that affect whether the iterative solution is considered valid for the system of equations. A smaller `atol` may lead to a more accurate solution, but it may also require more iterations to converge, while a larger `atol` may lead to a quicker solution but with less accuracy.

### Curious: Performance of iterative versus QR decomposition methods for solving linear equations?
So I'm curious about how the performance of our iterative methods compares to the QR decomposition method for solving linear equations. Let's set up a benchmarking experiment (using [the `@benchmark` macro exported from the `BenchmarkTools.jl` package](https://github.com/JuliaCI/BenchmarkTools.jl)) to compare the time taken by each method to solve the same system of equations.

Let's start with the QR decomposition method and then compare it to the iterative methods.

In [43]:
@benchmark begin
    Q,R = qr($A);
    x_qr = R \ (transpose(Q) * $b);
end

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  144.208 μs …  13.692 ms  ┊ GC (min … max): 0.00% … 98.59%
 Time  (median):     159.375 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   177.373 μs ± 247.911 μs  ┊ GC (mean ± σ):  7.34% ±  8.28%

  ▆█▆▄▂▁                                                        ▁
  ████████▇▇▇▇▆▇▆▆▅▅▅▅▅▅▃▁▄▄▃▃▃▃▃▄▁▃▃▁▁▃▁▁▁▃▁▁▁▁▁▃▃▃▄▃▁▃▃▁▃▃▃▄▅ █
  144 μs        Histogram: log(frequency) by time        729 μs <

 Memory estimate: 251.98 KiB, allocs estimate: 17.

Next, let's look at the performance of our iterative methods. Use the same setup as above to get an apples to apples comparison of the time and memory taken our iterative method versus the QR decomposition method.

In [44]:
let
    
    # initialize -
    number_of_rows = size(A, 1);
    maximum_number_of_iterations = 25000; # maximum number of iterations for the iterative method
    tolerance = 1e-12;
    algorithm = GaussSeidelMethod(); # change this to Jacobi(), GaussSeidelMethod() or SuccessiveOverRelaxationMethod() to test other algorithms
    ω = 0.6; # relaxation factor, only used for SOR
    xₒ = 0.1*ones(number_of_rows); # initial solution guess xₒ

     # call the solve method with the appropriate parameters -
    @benchmark VLDataScienceMachineLearningPackage.solve($A, $b, $xₒ; algorithm = $algorithm, 
        maxiterations = $maximum_number_of_iterations, 
        ϵ = $tolerance, 
        ω = $ω # only used for SOR
    );
end

BenchmarkTools.Trial: 45 samples with 1 evaluation per sample.
 Range (min … max):  100.126 ms … 287.121 ms  ┊ GC (min … max): 0.00% … 64.39%
 Time  (median):     107.655 ms               ┊ GC (median):    5.12%
 Time  (mean ± σ):   112.652 ms ±  27.483 ms  ┊ GC (mean ± σ):  8.54% ± 10.18%

  ▆ █                                                            
  ███▇▄▄▅▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▁
  100 ms           Histogram: frequency by time          287 ms <

 Memory estimate: 104.95 MiB, allocs estimate: 232759.

__Wow!__ We are slow (and use a lot of memory) compared to the QR decomposition method for solving linear equations. Hmmm. Maybe we should consider doing some refactoring! But that is a task for another day. For now, let's just note that the QR decomposition method is significantly faster and more memory efficient than our iterative methods for solving linear equations in this case.
___

## Summary: What Did We Learn?

So what's the takeaway from all this number crunching? We've just put direct and iterative methods head-to-head using a classic test problem, and the results tell an interesting story.

**The manufactured solution approach** turned out to be brilliant for validation. By working backwards from a known answer, we could actually measure how well our methods performed rather than just hoping they were correct.

**QR decomposition** emerged as the clear winner in terms of speed and accuracy for this problem. It's a direct method that just works - you factor the matrix once and you're done. No worrying about convergence or tuning parameters.

**Iterative methods** showed us the importance of parameters like tolerance and maximum iterations. They can be more memory-efficient for large sparse systems, but they require careful attention to convergence behavior. The diagonal dominance check wasn't just academic - it actually guaranteed convergence.

**The performance comparison** was eye-opening. Sometimes the "fancier" method isn't always the faster one, especially when the implementation isn't optimized. This reinforces why benchmarking is so important in computational work.

The key insight? There's no one-size-fits-all solution. For small to medium dense systems like ours, QR decomposition is hard to beat. But for huge sparse systems, iterative methods might be your only practical option. Understanding both approaches gives you the tools to pick the right method for the job.